## Imports

In [1]:
import copy
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from config import config
import os
import sys
current_dir = os.getcwd()
path = "C:\\Users\\eirik\\Documents\\Master\\ISLBBNN\\islbbnn"
# path = "C:\\you\\path\\to\\islbbnn\\folder\\here"
os.chdir(path)
import plot_functions as pf
import pipeline_functions as pip_func
sys.path.append('networks')
from flow_net import BayesianNetwork
import torch.nn.functional as F

os.chdir(current_dir) # set the working directory back to this one 

CPUs are used!


# Problem description

We will work with the __abalone__ dataset, where we wan to predict the number of rings in abalones, which should correspond to the age. 

# Batch size and parameters

In [2]:
# define parameters
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
post_train_epochs = config['post_train_epochs']
dim = config['hidden_dim']
num_transforms = config['num_transforms']
n_nets = config['n_nets']
lr = config['lr']
class_problem = config["class_problem"]
verbose = config['verbose']
save_res = config['save_res']
patience = config['patience']
SAMPLES = 1




#---------DATA------------
df = pd.read_csv("abalone.csv")
df = pd.get_dummies(df, drop_first=True)*1
#df = df.drop(["Sex"])
X_original = df.loc[:,df.columns != "Rings"].values
y_original = df.loc[:,df.columns == "Rings"].values.T[0]
n, p = X_original.shape  # need this to get p 

print(n,p)

# Define BATCH sizes
BATCH_SIZE = int((n*0.8)/5)
TEST_BATCH_SIZE = int(n*0.10) 
VAL_BATCH_SIZE = int(n*0.10)

TRAIN_SIZE = int((n*0.80)/5)
TEST_SIZE = int(n*0.10)
VAL_SIZE = int(n*0.10)

NUM_BATCHES = TRAIN_SIZE/BATCH_SIZE

assert (TRAIN_SIZE % BATCH_SIZE) == 0
assert (TEST_SIZE % TEST_BATCH_SIZE) == 0

4177 9


# Sigmoid based network

## Seperate a test set for later

In [3]:
# Split keep some of the data for validation after training
X, X_test, y, y_test = train_test_split(
    copy.deepcopy(X_original), copy.deepcopy(y_original), test_size=0.10, random_state=42)#, stratify=y)

test_dat = torch.tensor(np.column_stack((X_test,y_test)),dtype = torch.float32)

## Train, validate, and test network

In [4]:
# select the device and initiate model

# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "mps")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

all_nets = {}
metrics_several_runs = []
metrics_median_several_runs = []
for ni in range(n_nets):
    post_train = False
    print('network', ni)
    # Initate network
    torch.manual_seed(ni+42)
    net = BayesianNetwork(dim, p, HIDDEN_LAYERS, classification=class_problem, num_transforms=num_transforms).to(DEVICE)
    alphas = pip_func.get_alphas_numpy(net)
    nr_weights = np.sum([np.prod(a.shape) for a in alphas])
    print(nr_weights)

    optimizer = optim.Adam(net.parameters(), lr=lr)
    
    all_nll = []
    all_loss = []

    # Split into training and test set
    X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=1/9, random_state=ni)#, stratify=y)
            
    train_dat = torch.tensor(np.column_stack((X_train,y_train)),dtype = torch.float32)
    val_dat = torch.tensor(np.column_stack((X_val,y_val)),dtype = torch.float32)
    
    # Train network
    counter = 0
    highest_acc = 0
    best_model = copy.deepcopy(net)
    for epoch in range(epochs + post_train_epochs):
        if verbose:
            print(epoch)
        nll, loss = pip_func.train(net, train_dat, optimizer, BATCH_SIZE, NUM_BATCHES, p, DEVICE, nr_weights, post_train=post_train)
        nll_val, loss_val, ensemble_val = pip_func.val(net, val_dat, DEVICE, verbose=verbose, reg=(not class_problem), post_train=post_train)
        if ensemble_val >= highest_acc:
            counter = 0
            highest_acc = ensemble_val
            best_model = copy.deepcopy(net)
        else:
            counter += 1
        
        all_nll.append(nll)
        all_loss.append(loss)

        if epoch == epochs-1:
            post_train = True   # Post-train --> use median model 
            for name, param in net.named_parameters():
                for i in range(HIDDEN_LAYERS+1):
                    #if f"linears{i}.lambdal" in name:
                    if f"linears.{i}.lambdal" in name:
                        param.requires_grad_(False)

        if counter >= patience:
            break
        
    all_nets[ni] = net 
    # Results
    metrics, metrics_median = pip_func.test_ensemble(all_nets[ni],test_dat,DEVICE,SAMPLES=10, reg=(not class_problem)) # Test same data 10 times to get average 
    metrics_several_runs.append(metrics)
    metrics_median_several_runs.append(metrics_median)
    pf.run_path_graph(all_nets[ni], threshold=0.5, save_path=f"path_graphs/flow/prob/test{ni}_sigmoid", show=verbose)

if verbose:
    print(metrics)
m = np.array(metrics_several_runs)
m_median = np.array(metrics_median_several_runs)

network 0
789
0
loss 12172.0322265625
nll 217.06808471679688
density 0.9821754566314707

val_loss: 35600.7383, val_nll: 23648.6582, val_ensemble: -4.6461, used_weights_median: 789

1
loss 12053.1142578125
nll 4.253060340881348
density 0.9813094516821839

val_loss: 26983.6406, val_nll: 14910.5615, val_ensemble: -2.5599, used_weights_median: 789

2
loss 12096.234375
nll 59.06257247924805
density 0.98030270067609

val_loss: 16961.3867, val_nll: 4918.1714, val_ensemble: -0.1742, used_weights_median: 789

3
loss 12062.98046875
nll 62.70619201660156
density 0.979225008811032

val_loss: 15531.4746, val_nll: 3529.0630, val_ensemble: 0.1574, used_weights_median: 789

4
loss 11963.7705078125
nll 18.50667953491211
density 0.9780718745387552

val_loss: 16324.9141, val_nll: 4397.7358, val_ensemble: -0.0500, used_weights_median: 789

5
loss 11883.5224609375
nll 33.99626159667969
density 0.9768254818874016

val_loss: 14728.8750, val_nll: 2897.4905, val_ensemble: 0.3082, used_weights_median: 789

6
lo

## Attain weigth graph

In [5]:
pf.run_path_graph_weight(net, save_path="path_graphs/flow/weight/temp_sigmoid", show=True, flow=True)

# ReLU based network

## Seperate a test set for later

In [6]:
# Split keep some of the data for validation after training
X, X_test, y, y_test = train_test_split(
    copy.deepcopy(X_original), copy.deepcopy(y_original), test_size=0.10, random_state=42)#, stratify=y)

test_dat = torch.tensor(np.column_stack((X_test,y_test)),dtype = torch.float32)

## Train, validate, and test network

In [7]:
# select the device and initiate model

# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "mps")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

all_nets = {}
metrics_several_runs = []
metrics_median_several_runs = []
for ni in range(n_nets):
    post_train = False
    print('network', ni)
    # Initate network
    torch.manual_seed(ni+42)
    #---------------------------
    # DIFFERENCE IS IN act_func=F.relu part
    net = BayesianNetwork(dim, p, HIDDEN_LAYERS, classification=class_problem, num_transforms=num_transforms, act_func=F.relu).to(DEVICE)
    #---------------------------
    alphas = pip_func.get_alphas_numpy(net)
    nr_weights = np.sum([np.prod(a.shape) for a in alphas])
    print(nr_weights)

    optimizer = optim.Adam(net.parameters(), lr=lr)
    
    all_nll = []
    all_loss = []

    # Split into training and test set
    X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=1/9, random_state=ni)#, stratify=y)
            
    train_dat = torch.tensor(np.column_stack((X_train,y_train)),dtype = torch.float32)
    val_dat = torch.tensor(np.column_stack((X_val,y_val)),dtype = torch.float32)
    
    # Train network
    counter = 0
    highest_acc = 0
    best_model = copy.deepcopy(net)
    for epoch in range(epochs + post_train_epochs):
        if verbose:
            print(epoch)
        nll, loss = pip_func.train(net, train_dat, optimizer, BATCH_SIZE, NUM_BATCHES, p, DEVICE, nr_weights, post_train=post_train)
        nll_val, loss_val, ensemble_val = pip_func.val(net, val_dat, DEVICE, verbose=verbose, reg=(not class_problem), post_train=post_train)
        if ensemble_val >= highest_acc:
            counter = 0
            highest_acc = ensemble_val
            best_model = copy.deepcopy(net)
        else:
            counter += 1
        
        all_nll.append(nll)
        all_loss.append(loss)

        if epoch == epochs-1:
            post_train = True   # Post-train --> use median model 
            for name, param in net.named_parameters():
                for i in range(HIDDEN_LAYERS+1):
                    #if f"linears{i}.lambdal" in name:
                    if f"linears.{i}.lambdal" in name:
                        param.requires_grad_(False)

        if counter >= patience:
            break
        
    all_nets[ni] = net 
    # Results
    metrics, metrics_median = pip_func.test_ensemble(all_nets[ni],test_dat,DEVICE,SAMPLES=10, reg=(not class_problem)) # Test same data 10 times to get average 
    metrics_several_runs.append(metrics)
    metrics_median_several_runs.append(metrics_median)
    pf.run_path_graph(all_nets[ni], threshold=0.5, save_path=f"path_graphs/flow/prob/test{ni}_relu", show=verbose)

if verbose:
    print(metrics)
m = np.array(metrics_several_runs)
m_median = np.array(metrics_median_several_runs)

network 0
789
0
loss 12174.0927734375
nll 248.2642364501953
density 0.9821727177640482

val_loss: 45544.1797, val_nll: 33634.0039, val_ensemble: -7.0301, used_weights_median: 789

1
loss 11921.732421875
nll 8.852783203125
density 0.9813274006124383

val_loss: 74361.1250, val_nll: 62428.3281, val_ensemble: -13.9047, used_weights_median: 789

2
loss 11935.6796875
nll 69.49442291259766
density 0.9803629929272999

val_loss: 23034.4316, val_nll: 11159.3125, val_ensemble: -1.6643, used_weights_median: 789

3
loss 11920.8994140625
nll 76.58489990234375
density 0.9794031689073347

val_loss: 21669.9062, val_nll: 9827.5352, val_ensemble: -1.3463, used_weights_median: 789

4
loss 11788.1826171875
nll 2.9484951496124268
density 0.9784140875433184

val_loss: 19778.3418, val_nll: 8008.1611, val_ensemble: -0.9119, used_weights_median: 789

5
loss 11716.330078125
nll 0.6168987154960632
density 0.977338501104686

val_loss: 17431.3750, val_nll: 5729.6670, val_ensemble: -0.3680, used_weights_median: 789


## Attain weight graph

In [8]:
pf.run_path_graph_weight(net, save_path="path_graphs/flow/weight/temp_relu", show=True, flow=True)

In [9]:
net.eval()
net(train_dat[0:7, :-1], ensemble=False)

tensor([[ 9.3437],
        [ 6.0271],
        [ 7.3197],
        [ 8.2420],
        [ 9.2225],
        [ 6.3988],
        [10.5793]], grad_fn=<AddBackward0>)

In [10]:
train_dat

tensor([[ 0.5400,  0.4350,  0.1450,  ...,  1.0000,  0.0000, 17.0000],
        [ 0.4300,  0.3250,  0.0900,  ...,  1.0000,  0.0000,  7.0000],
        [ 0.4600,  0.3500,  0.1100,  ...,  1.0000,  0.0000,  9.0000],
        ...,
        [ 0.6200,  0.4750,  0.1600,  ...,  0.0000,  0.0000,  9.0000],
        [ 0.6700,  0.5450,  0.1600,  ...,  0.0000,  0.0000, 15.0000],
        [ 0.6350,  0.5050,  0.1900,  ...,  1.0000,  0.0000, 17.0000]])